In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
import os
!pip install gitpython
import git
git.Repo.clone_from('https://github.com/gregorbachmann/scaling_mlps.git', '/kaggle/working/sample')
import sys

In [2]:
sys.path.append('/kaggle/working/sample')

In [3]:
## Check if accelerator CUDA
torch.cuda.is_available()

True

In [4]:
import json
import torch

from sample.models import get_architecture
from sample.utils.parsers import get_finetune_parser
from sample.utils.config import config_to_name, model_from_config


# Path to pre-trained checkpoints and the corresponding config file (e.g. the checkpoints downloaded from given link)
checkpoint_path = '/kaggle/input/data-test-5/epoch_800'
config_path = '/kaggle/input/data-test-5/config.txt'

model, architecture, crop_resolution, norm = model_from_config(checkpoint_path)
with open(config_path, 'r') as f:
        config = json.load(f)

model = get_architecture(**config).cuda()
print('Loading checkpoint', checkpoint_path)

params = {
    k: v
    for k, v in torch.load(checkpoint_path).items()
}

# Load pre-trained parameters 
print('Load_state output', model.load_state_dict(params, strict=False))

Loading checkpoint /kaggle/input/data-test-5/epoch_800
Load_state output <All keys matched successfully>


In [5]:
model.eval()

BottleneckMLP(
  (act): GELU(approximate='none')
  (dropout): Identity()
  (linear_in): Linear(in_features=12288, out_features=512, bias=True)
  (linear_out): Linear(in_features=512, out_features=11230, bias=True)
  (blocks): ModuleList(
    (0-5): 6 x BottleneckBlock(
      (block): Sequential(
        (0): Linear(in_features=512, out_features=2048, bias=True)
        (1): GELU(approximate='none')
        (2): Linear(in_features=2048, out_features=512, bias=True)
      )
    )
  )
  (layernorms): ModuleList(
    (0-5): 6 x LayerNorm((512,), eps=1e-05, elementwise_affine=True)
  )
)

In [6]:
## Next step: evaluate the models performance on our adversarial data 
### As preprocessing for now just the basics so it doesnt errors
### Consider normalizing the with mean and variance, 
### left for later to get the important stuff out of the way first

## 
import torchvision.datasets as dataset
from torch.utils.data import DataLoader
from torchvision import transforms
root="/kaggle/input/natural-adversarial-examples-imageneta/imagenet-a/imagenet-a"

torch.manual_seed(1)
transform = transforms.Compose([
    # Add other transformations here
    transforms.Resize((64,64)),         ## Models were pretrained on input size 3x64x64
    #transforms.CenterCrop(64),
    transforms.ToTensor() 
])
img_ds=dataset.ImageFolder(root,transform)
img_dl=DataLoader(img_ds,batch_size=128,shuffle=False)

In [7]:
for images,labels in img_dl:
    print(images[123],'\n',labels[123].item()) ## Just visualisation, remove if in the way
    break
    
ims,lbs=images.cuda(),labels.cuda()
ims = torch.reshape(ims, (ims.shape[0], -1)) ## Need to be flattened 

preds = model(ims)
lbs.shape,preds.shape                       ## (128), (128,11230)
## Model was pretrained on the 21kImageNet data which have 11230 classes,  
## We should probably discuss this 
print(model.dim_out)

tensor([[[0.6353, 0.5765, 0.6392,  ..., 0.6235, 0.4588, 0.4039],
         [0.6784, 0.6039, 0.6118,  ..., 0.5412, 0.4392, 0.3529],
         [0.7059, 0.6157, 0.5882,  ..., 0.5255, 0.4784, 0.4392],
         ...,
         [0.6549, 0.5686, 0.6039,  ..., 0.4275, 0.4392, 0.4510],
         [0.7216, 0.7529, 0.8314,  ..., 0.2471, 0.2667, 0.2784],
         [0.6392, 0.6667, 0.7569,  ..., 0.1137, 0.1216, 0.1255]],

        [[0.6353, 0.5961, 0.6627,  ..., 0.7216, 0.5529, 0.5059],
         [0.6980, 0.6549, 0.6353,  ..., 0.6078, 0.5333, 0.4510],
         [0.7255, 0.6392, 0.6078,  ..., 0.6078, 0.6000, 0.5725],
         ...,
         [0.6118, 0.5137, 0.5529,  ..., 0.4431, 0.4588, 0.4667],
         [0.6549, 0.6863, 0.7725,  ..., 0.2392, 0.2588, 0.2745],
         [0.5647, 0.5922, 0.6824,  ..., 0.0863, 0.0902, 0.0980]],

        [[0.5294, 0.4980, 0.5765,  ..., 0.7255, 0.5451, 0.5176],
         [0.5686, 0.5020, 0.5216,  ..., 0.5725, 0.5294, 0.4706],
         [0.6157, 0.5216, 0.5216,  ..., 0.5765, 0.6039, 0.

In [8]:
## 1 epsilon sharing - optimierung genau anschauen 
### Regularization in the loss, for each layer a norm for diffference of matrices 
### How to get the weights into the loss

## 2 Wir erzwingen sharen und schuen ob sparsity passiert by itself 
### Einfach dann mal lambda=100000 und brute force 
### 